In [91]:
# import the proper packages
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

## Gourmet Gizmos

Assume that 3 plant locations are up and running so you don't have to worry about fixed costs.

In [92]:
# Start with Gourmet Gizmos
# Get data
# Gourmet Gizmos demand is in a sheet called 'gg_demand'
gg_demand = pd.read_excel('KitchenCompanies.xlsx', sheet_name='gg_demand', index_col=0)
gg_demand

,demand
city,
Atlanta,10
Boston,8
Chicago,14


In [93]:
# Convert demand to a dictionary
gg_demand = gg_demand.to_dict()['demand']
gg_demand

{'Atlanta': 10, 'Boston': 8, 'Chicago': 14}

In [94]:
# Create the markets as a list
markets = list(gg_demand.keys())
markets

['Atlanta', 'Boston', 'Chicago']

In [95]:
# Get data
# the capacity, variable, and fixed costs are in a sheet labeled 'gg_costs_and_cap'
gg_data = pd.read_excel('KitchenCompanies.xlsx', sheet_name='gg_costs_and_cap', index_col=0)
gg_data

,Atlanta,Boston,Chicago,monthly_capacity,monthly_fc
supply_city,,,,,
Bethesda,1675,400,685,18,7650
Memphis,380,1355,543,22,4100
Kansas_City,922,1646,700,31,2200


In [96]:
# Create the plants as a list
plants = gg_data.index.tolist()
plants

['Bethesda', 'Memphis', 'Kansas_City']

In [97]:
# Create a dictionary to hold the capacity for each plant
gg_capacity = gg_data['monthly_capacity'].to_dict()
gg_capacity

{'Bethesda': 18, 'Memphis': 22, 'Kansas_City': 31}

In [98]:
# Create a dictionary to hold the fixed cost for each plant
gg_fc = gg_data['monthly_fc'].to_dict()
gg_fc

{'Bethesda': 7650, 'Memphis': 4100, 'Kansas_City': 2200}

In [99]:
# Get data
# Transportation costs are in our gg_costs_and_cap DataFrame
gg_transp = gg_data[['Atlanta', 'Boston', 'Chicago']]
gg_transp

,Atlanta,Boston,Chicago
supply_city,,,
Bethesda,1675,400,685
Memphis,380,1355,543
Kansas_City,922,1646,700


In [100]:
# Convert costs to a list of lists
gg_transp_costs = gg_transp.values.tolist()
gg_transp_costs

[[1675, 400, 685], [380, 1355, 543], [922, 1646, 700]]

In [101]:
# Create the model instance and set to minimize
model = gp.Model('Gourmet_Gizmos')
# Set it to minimize
model.modelSense = GRB.MINIMIZE

In [102]:
# Create the flow variables from plants to markets
flow = model.addVars(plants, markets, obj=gg_transp_costs, name='flow')

# Update the model and display
model.update()
model.display()

Minimize
1675.0 flow[Bethesda,Atlanta] + 400.0 flow[Bethesda,Boston]
+ 685.0 flow[Bethesda,Chicago] + 380.0 flow[Memphis,Atlanta]
+ 1355.0 flow[Memphis,Boston] + 543.0 flow[Memphis,Chicago]
+ 922.0 flow[Kansas_City,Atlanta] + 1646.0 flow[Kansas_City,Boston]
+ 700.0 flow[Kansas_City,Chicago]
Subject To


/tmp/ipykernel_4146/3080150752.py:6: DeprecationWarning: Model.display() is deprecated
  model.display()


In [103]:
# Make sure you meet demand for each market
for m in markets:
    model.addConstr(gp.quicksum(flow[p, m] for p in plants) == gg_demand[m], name=f"demand_{m}")

# Update and display
model.update()
model.display()


Minimize
1675.0 flow[Bethesda,Atlanta] + 400.0 flow[Bethesda,Boston]
+ 685.0 flow[Bethesda,Chicago] + 380.0 flow[Memphis,Atlanta]
+ 1355.0 flow[Memphis,Boston] + 543.0 flow[Memphis,Chicago]
+ 922.0 flow[Kansas_City,Atlanta] + 1646.0 flow[Kansas_City,Boston]
+ 700.0 flow[Kansas_City,Chicago]
Subject To
demand_Atlanta: flow[Bethesda,Atlanta] + flow[Memphis,Atlanta] +
 flow[Kansas_City,Atlanta] = 10
demand_Boston: flow[Bethesda,Boston] + flow[Memphis,Boston] + flow[Kansas_City,Boston]
 = 8
demand_Chicago: flow[Bethesda,Chicago] + flow[Memphis,Chicago] +
 flow[Kansas_City,Chicago] = 14


/tmp/ipykernel_4146/4152660249.py:7: DeprecationWarning: Model.display() is deprecated
  model.display()


In [104]:
# Make sure stay under capacity at plants
for p in plants:
    model.addConstr(gp.quicksum(flow[p, m] for m in markets) <= gg_capacity[p], name=f"capacity_{p}")

# Update and display
model.update()
model.display()

Minimize
1675.0 flow[Bethesda,Atlanta] + 400.0 flow[Bethesda,Boston]
+ 685.0 flow[Bethesda,Chicago] + 380.0 flow[Memphis,Atlanta]
+ 1355.0 flow[Memphis,Boston] + 543.0 flow[Memphis,Chicago]
+ 922.0 flow[Kansas_City,Atlanta] + 1646.0 flow[Kansas_City,Boston]
+ 700.0 flow[Kansas_City,Chicago]
Subject To
demand_Atlanta: flow[Bethesda,Atlanta] + flow[Memphis,Atlanta] +
 flow[Kansas_City,Atlanta] = 10
demand_Boston: flow[Bethesda,Boston] + flow[Memphis,Boston] + flow[Kansas_City,Boston]
 = 8
demand_Chicago: flow[Bethesda,Chicago] + flow[Memphis,Chicago] +
 flow[Kansas_City,Chicago] = 14
capacity_Bethesda: flow[Bethesda,Atlanta] + flow[Bethesda,Boston] +
 flow[Bethesda,Chicago] <= 18
capacity_Memphis: flow[Memphis,Atlanta] + flow[Memphis,Boston] + flow[Memphis,Chicago]
 <= 22
capacity_Kansas_City: flow[Kansas_City,Atlanta] + flow[Kansas_City,Boston] +
 flow[Kansas_City,Chicago] <= 31


/tmp/ipykernel_4146/1222814154.py:7: DeprecationWarning: Model.display() is deprecated
  model.display()


In [105]:
# Solve
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x925f4a86
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+02, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 3e+01]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4602000e+04   2.000000e+00   0.000000e+00      0s
       1    1.4886000e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.488600000e+04


In [106]:
# Print answer
print(f"Total Cost: ${model.objVal:,.2f}")
for var in model.getVars():
    if var.x > 0:
        print(f"{var.varName} = {var.x}")

Total Cost: $14,886.00
flow[Bethesda,Boston] = 8.0
flow[Bethesda,Chicago] = 2.0
flow[Memphis,Atlanta] = 10.0
flow[Memphis,Chicago] = 12.0


## Culinary Compacts

Assume the two supply locations are already up and running so you do not need to incorporate the fixed costs.

In [107]:
# Culinary Compacts
# Get data
# Culinary Compacts demand is in a sheet called 'cc_demand'
cc_demand = pd.read_excel('KitchenCompanies.xlsx', sheet_name='cc_demand', index_col=0)
cc_demand

,demand
city,
Denver,6
Omaha,7
Seattle,11


In [108]:
# Convert demand to a dictionary
cc_demand = cc_demand.to_dict()['demand']
cc_demand

{'Denver': 6, 'Omaha': 7, 'Seattle': 11}

In [109]:
# Create the markets as a list
cc_markets = list(cc_demand.keys())
cc_markets

['Denver', 'Omaha', 'Seattle']

In [110]:
# Get data
# the capacity, variable, and fixed costs are in a sheet labeled 'gg_costs_and_cap'
cc_data = pd.read_excel('KitchenCompanies.xlsx', sheet_name='cc_costs_and_cap', index_col=0)
cc_data

,Denver,Omaha,Seattle,monthly_capacity,monthly_fc
supply_city,,,,,
Fort_Collins,100,495,1200,24,3500
Provo,500,950,800,27,5000


In [111]:
# Create the plants as a list
cc_plants = cc_data.index.tolist()
cc_plants

['Fort_Collins', 'Provo']

In [112]:
# Create a dictionary to hold the capacity for each plant
cc_capacity = cc_data['monthly_capacity'].to_dict()
cc_capacity

{'Fort_Collins': 24, 'Provo': 27}

In [113]:
# Create a dictionary to hold the fixed cost for each plant
cc_fixed_costs = cc_data['monthly_fc'].to_dict()
cc_fixed_costs

{'Fort_Collins': 3500, 'Provo': 5000}

In [114]:
# Get data
# Transportation costs are in our cc_costs_and_cap DataFrame
cc_transp_cost = cc_data[['Denver', 'Omaha', 'Seattle']]
cc_transp_cost

,Denver,Omaha,Seattle
supply_city,,,
Fort_Collins,100,495,1200
Provo,500,950,800


In [115]:
# Convert costs to a list of lists
cc_transp_cost = cc_transp_cost.values.tolist()
cc_transp_cost

[[100, 495, 1200], [500, 950, 800]]

In [120]:
# Create the model instance and set to minimize
cc_model = gp.Model('Culinary_Compacts')
cc_model.ModelSense = GRB.MINIMIZE

# Create the flow variables from plants to markets
cc_flow = cc_model.addVars(cc_plants, cc_markets, obj=cc_transp_cost, name='flow')

# Make sure you meet demand for each market
for market in cc_markets:
    cc_model.addConstr(gp.quicksum(cc_flow[p, market] for p in cc_plants) >= cc_demand[market], name=f"demand_{market}")

# Make sure stay under capacity at plants
for plant in cc_plants:
    cc_model.addConstr(gp.quicksum(cc_flow[plant, m] for m in cc_markets) <= cc_capacity[plant], name=f"capacity_{plant}")

# Update and display
cc_model.update()
cc_model.display()

Minimize
100.0 flow[Fort_Collins,Denver] + 495.0 flow[Fort_Collins,Omaha]
+ 1200.0 flow[Fort_Collins,Seattle] + 500.0 flow[Provo,Denver] + 950.0 flow[Provo,Omaha]
+ 800.0 flow[Provo,Seattle]
Subject To
  demand_Denver: flow[Fort_Collins,Denver] + flow[Provo,Denver] >= 6
  demand_Omaha: flow[Fort_Collins,Omaha] + flow[Provo,Omaha] >= 7
  demand_Seattle: flow[Fort_Collins,Seattle] + flow[Provo,Seattle] >= 11
capacity_Fort_Collins: flow[Fort_Collins,Denver] + flow[Fort_Collins,Omaha] +
 flow[Fort_Collins,Seattle] <= 24
  capacity_Provo: flow[Provo,Denver] + flow[Provo,Omaha] + flow[Provo,Seattle] <= 27


/tmp/ipykernel_4146/2362236160.py:18: DeprecationWarning: Model.display() is deprecated
  cc_model.display()


In [121]:
# solve
cc_model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xa7a90fb5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 3e+01]
Presolve time: 0.04s
Presolved: 5 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.400000e+01   0.000000e+00      0s
       3    1.2865000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.05 seconds (0.00 work units)
Optimal objective  1.286500000e+04


In [122]:
# Print answer
print(f"Total Cost: ${cc_model.objVal:,.2f}")
for var in cc_model.getVars():
    if var.x > 0:
        print(f"{var.varName} = {var.x}")

Total Cost: $12,865.00
flow[Fort_Collins,Denver] = 6.0
flow[Fort_Collins,Omaha] = 7.0
flow[Provo,Seattle] = 11.0


## Combined Company - Culinary Gizmos

Now we need to incorporate the fixed costs for each manufacturing plant when deciding the best design configuration.

In [125]:
# Culinary Gizmos - the combined company
# Get data
# Culinary Gizmos demand is in a sheet called 'combined_demand'
combined_demand = pd.read_excel('KitchenCompanies.xlsx', sheet_name='combined_demand', index_col=0)
display(combined_demand)

# Convert demand to a dictionary
combined_demand = combined_demand.to_dict()['demand']
display(combined_demand)

# Create the markets as a list
combined_markets = list(combined_demand.keys())
display(combined_markets)

,demand
city,
Atlanta,10
Boston,8
Chicago,14
Denver,6
Omaha,7
Seattle,11


{'Atlanta': 10,
 'Boston': 8,
 'Chicago': 14,
 'Denver': 6,
 'Omaha': 7,
 'Seattle': 11}

['Atlanta', 'Boston', 'Chicago', 'Denver', 'Omaha', 'Seattle']

In [126]:
# Get data
# the capacity, variable, and fixed costs are in a sheet labeled 'combined_costs_and_cap'
combined_data = pd.read_excel('KitchenCompanies.xlsx', sheet_name='combined_costs_and_cap', index_col=0)
display(combined_data)

,Atlanta,Boston,Chicago,Denver,Omaha,Seattle,monthly_capacity,monthly_fc
supply_city,,,,,,,,
Bethesda,1675,400,685,1630,1160,2800,18,7650
Memphis,380,1355,543,1045,665,2321,22,4100
Kansas_City,922,1646,700,508,311,1797,31,2200
Fort_Collins,1460,1940,970,100,495,1200,24,3500
Provo,1925,2400,1425,500,950,800,27,5000


In [128]:
# Create the plants as a list
combined_plants = combined_data.index.tolist()
display(combined_plants)

# Create a dictionary to hold the capacity for each plant
combined_capacity = combined_data['monthly_capacity'].to_dict()
display(combined_capacity)

# Create a dictionary to hold the fixed cost for each plant
combined_fixed_cost = combined_data['monthly_fc'].to_dict()
display(combined_fixed_cost)

# Get data
# Transportation costs are in our all_costs_and_cap DataFrame
combined_transp_cost = combined_data[['Atlanta', 'Boston', 'Chicago', 'Denver', 'Omaha', 'Seattle']]
display(combined_transp_cost)

# Convert costs to a list of lists
combined_transp_cost = combined_transp_cost.values.tolist()
display(combined_transp_cost)

['Bethesda', 'Memphis', 'Kansas_City', 'Fort_Collins', 'Provo']

{'Bethesda': 18,
 'Memphis': 22,
 'Kansas_City': 31,
 'Fort_Collins': 24,
 'Provo': 27}

{'Bethesda': 7650,
 'Memphis': 4100,
 'Kansas_City': 2200,
 'Fort_Collins': 3500,
 'Provo': 5000}

,Atlanta,Boston,Chicago,Denver,Omaha,Seattle
supply_city,,,,,,
Bethesda,1675,400,685,1630,1160,2800
Memphis,380,1355,543,1045,665,2321
Kansas_City,922,1646,700,508,311,1797
Fort_Collins,1460,1940,970,100,495,1200
Provo,1925,2400,1425,500,950,800


[[1675, 400, 685, 1630, 1160, 2800],
 [380, 1355, 543, 1045, 665, 2321],
 [922, 1646, 700, 508, 311, 1797],
 [1460, 1940, 970, 100, 495, 1200],
 [1925, 2400, 1425, 500, 950, 800]]

In [132]:
# Create the model instance and set to minimize
combined_model = gp.Model('Culinary_Gizmos')
combined_model.ModelSense = GRB.MINIMIZE

# Create the flow variables from plants to markets
combined_flow = combined_model.addVars(combined_plants, combined_markets, obj=combined_transp_cost, name='flow')

# Create the binary variables to open plants
open_plant = combined_model.addVars(combined_plants, vtype=GRB.BINARY, name='open')

# update
model.update()
# model.display()

# Make sure you meet demand for each market
for market in combined_markets:
    combined_model.addConstr(gp.quicksum(combined_flow[p, market] for p in combined_plants) >= combined_demand[market], name=f"demand_{market}")

# Make sure stay under copacity at plants
for plant in combined_plants:
    combined_model.addConstr(gp.quicksum(combined_flow[plant, m] for m in combined_markets) <= combined_capacity[plant] * open_plant[plant], name=f"capacity_{plant}")

# Update and display
model.update()
# model.display()

In [130]:
# solve
combined_model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 11 rows, 35 columns and 65 nonzeros
Model fingerprint: 0x44847c3a
Variable types: 30 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+00, 1e+01]
Presolve removed 0 rows and 5 columns
Presolve time: 0.01s
Presolved: 11 rows, 30 columns, 60 nonzeros
Variable types: 30 continuous, 0 integer (0 binary)

Root relaxation: objective 2.646300e+04, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    26463.000000 26463.0000  0.00%     -

In [131]:
# Print answer
print(f"Total Cost: ${combined_model.objVal:,.2f}")
for var in combined_model.getVars():
    if var.x > 0:
        print(f"{var.varName} = {var.x}")

Total Cost: $26,463.00
flow[Bethesda,Boston] = 8.0
flow[Bethesda,Chicago] = 2.0
flow[Memphis,Atlanta] = 10.0
flow[Memphis,Chicago] = 12.0
flow[Kansas_City,Omaha] = 7.0
flow[Fort_Collins,Denver] = 6.0
flow[Provo,Seattle] = 11.0
open[Bethesda] = 1.0
open[Memphis] = 1.0
open[Kansas_City] = 1.0
open[Fort_Collins] = 1.0
open[Provo] = 1.0
